### Import Libraries

In [1]:
import psycopg2
from psycopg2 import sql
import pandas as pd
from sqlalchemy import create_engine

### Import data

In [2]:
data = pd.read_csv("dataset/train.csv")

In [3]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
data.shape

(8693, 14)

In [5]:
data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [7]:
def correct_datatype(data):
    obj_cols = ["PassengerId","Cabin","Destination"]
    num_cols = ['Age', 'RoomService', 'FoodCourt','ShoppingMall', 'Spa', 'VRDeck']
    bool_cols = ["CryoSleep","Transported"]
    cat_cols = [col for col in data.columns if col not in obj_cols+num_cols+bool_cols]

    for col in data.columns:
        if col in bool_cols:
            data[col] = data[col].astype('bool')
        elif col in cat_cols:
            data[col] = data[col].astype('category')
        elif col in num_cols:
            data[col] = data[col].astype('float')

    return data

In [8]:
data = correct_datatype(data)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   PassengerId   8693 non-null   object  
 1   HomePlanet    8492 non-null   category
 2   CryoSleep     8693 non-null   bool    
 3   Cabin         8494 non-null   object  
 4   Destination   8511 non-null   object  
 5   Age           8514 non-null   float64 
 6   VIP           8490 non-null   category
 7   RoomService   8512 non-null   float64 
 8   FoodCourt     8510 non-null   float64 
 9   ShoppingMall  8485 non-null   float64 
 10  Spa           8510 non-null   float64 
 11  VRDeck        8505 non-null   float64 
 12  Name          8493 non-null   category
 13  Transported   8693 non-null   bool    
dtypes: bool(2), category(3), float64(6), object(3)
memory usage: 1.0+ MB


### Populate the database


In [10]:
engine = create_engine("< INSERT DATABASE URI >", echo = True)

In [11]:
data.to_sql("titanicspaceship", con = engine, if_exists='append')

2022-03-26 15:33:35,042 INFO sqlalchemy.engine.base.Engine select version()
2022-03-26 15:33:35,044 INFO sqlalchemy.engine.base.Engine {}
2022-03-26 15:33:35,472 INFO sqlalchemy.engine.base.Engine select current_schema()
2022-03-26 15:33:35,474 INFO sqlalchemy.engine.base.Engine {}
2022-03-26 15:33:35,905 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-03-26 15:33:35,907 INFO sqlalchemy.engine.base.Engine {}
2022-03-26 15:33:36,110 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-03-26 15:33:36,113 INFO sqlalchemy.engine.base.Engine {}
2022-03-26 15:33:36,341 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2022-03-26 15:33:36,342 INFO sqlalchemy.engine.base.Engine {}
2022-03-26 15:33:36,760 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

### Confirm database population

In [12]:
print(engine.execute("SELECT * FROM titanicspaceship").fetchone ())

2022-03-26 16:06:52,890 INFO sqlalchemy.engine.base.Engine SELECT * FROM titanicspaceship
2022-03-26 16:06:52,892 INFO sqlalchemy.engine.base.Engine {}
(0, '0001_01', 'Europa', False, 'B/0/P', 'TRAPPIST-1e', 39.0, 'false', 0.0, 0.0, 0.0, 0.0, 0.0, 'Maham Ofracculy', False)
